In [33]:
import polars as pl

In [34]:
# Checking the current file location

!dir

 Volume in drive C is Windows
 Volume Serial Number is 5E9C-93AA

 Directory of C:\Users\mp5667di\Desktop\DH_comparisons

09/04/2024  08:01 AM    <DIR>          .
09/04/2024  07:35 AM    <DIR>          ..
09/03/2024  11:44 AM                66 .gitattributes
09/04/2024  07:35 AM    <DIR>          .ipynb_checkpoints
09/04/2024  08:01 AM            21,976 comparing_league_wide_ERA.ipynb
09/04/2024  07:55 AM    <DIR>          Data
09/03/2024  11:44 AM                26 README.md
               3 File(s)         22,068 bytes
               4 Dir(s)  20,219,375,616 bytes free


In [35]:
pitching = pl.read_csv("./Data/lahman_1871-2023_csv/Pitching.csv")
pitching.head()

playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""aardsda01""",2004,1,"""SFN""","""NL""",1,0,11,0,0,0,0,32,20,8,1,10,5,0.417,6.75,0,0,2,0,61,5,8,0,1,1
"""aardsda01""",2006,1,"""CHN""","""NL""",3,0,45,0,0,0,0,159,41,24,9,28,49,0.214,4.08,0,1,1,0,225,9,25,1,3,2
"""aardsda01""",2007,1,"""CHA""","""AL""",2,1,25,0,0,0,0,97,39,23,4,17,36,0.3,6.4,3,2,1,0,151,7,24,2,1,1
"""aardsda01""",2008,1,"""BOS""","""AL""",4,2,47,0,0,0,0,146,49,30,4,35,49,0.268,5.55,2,3,5,0,228,7,32,3,2,4
"""aardsda01""",2009,1,"""SEA""","""AL""",3,6,73,0,0,0,38,214,49,20,4,34,80,0.19,2.52,3,2,0,0,296,53,23,2,1,2


In [36]:
# There are 7 types of leagues 

pitching['lgID'].unique().to_list()

['AA', 'AL', 'FL', 'UA', 'NA', 'PL', 'NL']

In [37]:
# Part1 - Q5

league_ERA = (pitching
              .select(['yearID','lgID','ER','IPouts'])
              .filter(pl.col('yearID') >= 1946)
              .filter(pl.col('lgID').is_in(['AL','NL']))
              .groupby(['yearID','lgID'])
              .agg(pl.col('ER').sum().alias('total_runs'),
                   pl.col('IPouts').sum().alias('total_outs')
                  )
              .with_columns(league_ERA = (pl.col('total_runs')/pl.col('total_outs'))*27)
              .sort(pl.col('yearID'))
             )
league_ERA.head()

yearID,lgID,total_runs,total_outs,league_ERA
i64,str,i64,i64,f64
1946,"""NL""",4202,33254,3.41174
1946,"""AL""",4306,33186,3.503345
1947,"""NL""",4956,32902,4.066987
1947,"""AL""",4560,33228,3.705309
1948,"""NL""",4814,32867,3.954666


In [38]:
# Part1 - Q6

DH_rule = (league_ERA
             .select(['yearID','lgID','total_runs','total_outs','league_ERA'])
             .with_columns(DH_rule = pl.when((pl.col('yearID') < 1973) & (pl.col('lgID') == 'AL'))
                                       .then('No')
                                       .otherwise('Yes')
                          )
             .with_columns(DH_rule = pl.when((pl.col('yearID') >= 2022) & (pl.col('yearID') == 2020) & (pl.col('lgID') == 'NL'))
                                       .then('Yes')
                                       .otherwise('No')
                          )
            )
DH_rule.head()

yearID,lgID,total_runs,total_outs,league_ERA,DH_rule
i64,str,i64,i64,f64,str
1946,"""NL""",4202,33254,3.41174,"""No"""
1946,"""AL""",4306,33186,3.503345,"""No"""
1947,"""NL""",4956,32902,4.066987,"""No"""
1947,"""AL""",4560,33228,3.705309,"""No"""
1948,"""NL""",4814,32867,3.954666,"""No"""


In [39]:
DH_rule.write_csv("./Data/DH_rule.csv")

In [40]:
batting = pl.read_csv("./Data/lahman_1871-2023_csv/Batting.csv")
batting.head()

playerID,yearID,stint,teamID,lgID,G,G_batting,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,G_old
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str
"""aardsda01""",2004,1,"""SFN""","""NL""",11,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null
"""aardsda01""",2006,1,"""CHN""","""NL""",45,null,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,null
"""aardsda01""",2007,1,"""CHA""","""AL""",25,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null
"""aardsda01""",2008,1,"""BOS""","""AL""",47,null,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,null
"""aardsda01""",2009,1,"""SEA""","""AL""",73,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null


In [41]:
# Part2 

SLG = (batting
      .select(['yearID','lgID','AB','H','2B','3B','HR'])
      .filter((pl.col('yearID') >= 1946) &
               (pl.col('lgID').is_in(['AL','NL']))
             )
      .groupby(['yearID','lgID'])
      .agg((pl.col('H').sum() + pl.col('2B').sum()*2 + pl.col('3B').sum()*3 + pl.col('HR').sum()*4).alias('numerator'),
           (pl.col('AB').sum()).alias('denominator')
          )
      .with_columns(SLG = pl.col('numerator')/pl.col('denominator'))
      .sort(pl.col('yearID'))
      )
SLG.head()

yearID,lgID,numerator,denominator,SLG
i64,str,i64,i64,f64
1946,"""NL""",17660,42094,0.419537
1946,"""AL""",18262,42239,0.432349
1947,"""NL""",19704,42434,0.464345
1947,"""AL""",18107,42002,0.431099
1948,"""NL""",19234,42256,0.455178


In [42]:
SLG.write_csv("./data/SLG.csv")